**Simulation Code for FIFO**

In [ ]:
import numpy as np
from collections import deque

# A few helpful functions
def average(list):
    return(sum(list[1:])/(len(list) - 1))

def U():
    return float(np.random.uniform(size=1))

def poisson_execution_time(lamda):
    X = - float(np.log(U())) * lamda
    return X

def next_system_arrival():
    lamda = 1000/0.9
    return poisson_execution_time(lamda)

def next_event_station(arrival, CPU, DISK):
    if CPU == None:
        if DISK != None and DISK < arrival:
            return "DISK"
        else:
            return "arrival"
    elif DISK == None:
        if CPU != None and CPU < arrival:
            return "CPU"
        else:
            return "arrival"
    else:
        if arrival < DISK and arrival < CPU:
            return "arrival"
        elif CPU < DISK:
            return "CPU"
        else:
            return "DISK"

# Initializations
CPUQueue = deque([])
queue_DISK = deque([])

current_job_ID_cpu = None
current_job_exec_start_time_cpu = None
current_job_exec_remaining_time_cpu = None

current_job_ID_disk = None
current_job_exec_start_time_disk = None
current_job_exec_remaining_time_disk = None

jobID = 1
clock = 0

# Initialize a few more structures for statistics
time_of_arrival_dict = {}
time_of_departure_dict = {}
disk_visits_per_job = {}
aggregate_response_time = []
cpu_unused_time_per_cycle = []
disk_unused_time_per_cycle = []
cycle_duration = []
job_count = []

# Unused time variables
cpu_unused_time = 0
disk_unused_time = 0

# CPU arrival time
time_of_next_arrival = next_system_arrival()
disk_unused_time = cpu_unused_time = 0
disk_last_empty_timestamp = cpu_last_empty_timestamp = 0

# CPU state
cpu_empty = True

# DISK state
disk_empty = True
regen_cycle = 0

# List of jobs per regenerative cycle
per_cycle_jobs_list = []
old_clock = 0

condition = True

while (condition and regen_cycle < 1000):
    
    # Start of the regen cycle
    # Everything is empty
    if cpu_empty and disk_empty:

        # Check the confidence interval every 20 cycles
        if (regen_cycle - 1) % 20 == 0 and regen_cycle != 1:

            # Calculate s ^ 2
            y_bar = average(aggregate_response_time)
            # Duration in number of jobs
            c_bar = average(job_count)
            n = regen_cycle - 1

            # sy represents sy^2
            sy = 0  
            sc = 0
            syc = 0
            for i in range(1, regen_cycle):
                sy += (aggregate_response_time[i] - y_bar) ** 2
                sc += (job_count[i] - c_bar) ** 2
                syc += (aggregate_response_time[i] -
                        y_bar) * (job_count[i] - c_bar)

            sy /= n - 1
            sc /= n - 1
            syc /= n - 1
            R = average(aggregate_response_time) / \
                average(job_count)
            s = sy - 2 * R * syc + R ** 2 * sc

            # z for 0.95
            z1_a_2 = 1.960
            confidence_interval = z1_a_2 * \
                float(np.sqrt(s)) / (c_bar * float(np.sqrt(n)))

            if confidence_interval / R < 0.1:
                condition = False
                break

        # Register cycle length
        duration = clock-old_clock
        cycle_duration.append(duration)
        old_clock = clock

        total_response_time = 0
        if per_cycle_jobs_list:
            for job in per_cycle_jobs_list:
                total_response_time += (time_of_departure_dict[job] - time_of_arrival_dict[job])
            aggregate_response_time.append(total_response_time)
            job_count.append(len(per_cycle_jobs_list))
        else:
            aggregate_response_time.append(0)
            job_count.append(0)

        if duration == 0:
            duration = 1

        # U_CPU data
        cpu_unused_time_per_cycle.append(cpu_unused_time)
        cpu_unused_time = 0

        # U_DISK data
        disk_unused_time_per_cycle.append(disk_unused_time)
        disk_unused_time = 0

        regen_cycle += 1
        clock = time_of_next_arrival
        cpu_unused_time += (clock - cpu_last_empty_timestamp)

        per_cycle_jobs_list = []

        # Begin the job's execution
        # Find job's processing time and assign the job to the CPU
        exec_time = poisson_execution_time(54)
        current_job_ID_cpu = jobID
        per_cycle_jobs_list.append(jobID)
        current_job_exec_start_time_cpu = clock
        current_job_exec_remaining_time_cpu = exec_time

        # Define new arrival time and increment number of jobs
        time_of_arrival_dict[jobID] = clock
        disk_visits_per_job[jobID] = 0
        time_of_next_arrival = clock + next_system_arrival()
        jobID += 1
        
        cpu_empty = False

    elif disk_empty:

        # Handle new arrival
        if next_event_station(time_of_next_arrival, current_job_exec_start_time_cpu + current_job_exec_remaining_time_cpu, None) == "arrival":
            clock = time_of_next_arrival

            # Initialize the job's processing time and assign the job to the CPU
            time_of_arrival_dict[jobID] = clock
            disk_visits_per_job[jobID] = 0
            per_cycle_jobs_list.append(jobID)

            # Place new job at CPU queue
            exec_time = poisson_execution_time(54)
            CPUQueue.append([jobID, exec_time])

            # Define new arrival time and increment number of jobs
            time_of_next_arrival = clock + next_system_arrival()
            cpu_empty = False
            
            jobID += 1

        else:
            clock = current_job_exec_start_time_cpu + current_job_exec_remaining_time_cpu

            # Job leaves the system with p = 1/19,
            # Job visits the disk with p = 18/19
            random = U()
            if (random <= 1/19):
                # Service time + exit time
                time_of_departure_dict[current_job_ID_cpu] = clock + 234
                if CPUQueue:
                    current_job_ID_cpu, current_job_exec_remaining_time_cpu = CPUQueue.popleft()
                    current_job_exec_start_time_cpu = clock
                elif not CPUQueue:
                    current_job_ID_cpu, current_job_exec_remaining_time_cpu, current_job_exec_start_time_cpu = None, None, None
                    cpu_last_empty_timestamp = clock
                    cpu_empty = True
                
            else:
                # Load job to the empty disk
                disk_visits_per_job[current_job_ID_cpu] += 1

                exec_time = poisson_execution_time(35)

                current_job_ID_disk = current_job_ID_cpu
                current_job_exec_remaining_time_disk = exec_time
                current_job_exec_start_time_disk = clock
                disk_unused_time += (clock - disk_last_empty_timestamp)
                disk_empty = False

                # Load new job to the CPU
                if CPUQueue:
                    current_job_ID_cpu, current_job_exec_remaining_time_cpu = CPUQueue.popleft()
                    current_job_exec_start_time_cpu = clock
                elif not CPUQueue:
                    current_job_ID_cpu, current_job_exec_remaining_time_cpu, current_job_exec_start_time_cpu = None, None, None
                    cpu_last_empty_timestamp = clock
                    cpu_empty = True

    elif cpu_empty:

        # Handle new arrival
        if next_event_station(time_of_next_arrival, None, current_job_exec_start_time_disk + current_job_exec_remaining_time_disk) == "arrival":
            clock = time_of_next_arrival
            time_of_arrival_dict[jobID] = clock
            disk_visits_per_job[jobID] = 0

            per_cycle_jobs_list.append(jobID)
            current_job_exec_remaining_time_cpu = poisson_execution_time(54)

            cpu_unused_time += (clock - cpu_last_empty_timestamp)
            current_job_ID_cpu = jobID
            current_job_exec_start_time_cpu = clock

            time_of_next_arrival = clock + next_system_arrival()
            cpu_empty = False
            
            jobID += 1

        else:
            clock = current_job_exec_start_time_disk + current_job_exec_remaining_time_disk
            # Load job to the empty CPU
            exec_time = poisson_execution_time(54)

            current_job_ID_cpu = current_job_ID_disk
            current_job_exec_remaining_time_cpu = exec_time
            current_job_exec_start_time_cpu = clock
            cpu_empty = False

            # Load new job to the DISK
            if queue_DISK:
                current_job_ID_disk, current_job_exec_remaining_time_disk = queue_DISK.popleft()
                current_job_exec_start_time_disk = clock
            elif not queue_DISK:
                current_job_ID_disk, current_job_exec_remaining_time_disk, current_job_exec_start_time_disk = None, None, None
                disk_last_empty_timestamp = clock
                disk_empty = True

    else:

        # Handle new arrival
        if next_event_station(time_of_next_arrival, current_job_exec_start_time_cpu + current_job_exec_remaining_time_cpu, current_job_exec_start_time_disk + current_job_exec_remaining_time_disk) == "arrival":
            clock = time_of_next_arrival
            time_of_arrival_dict[jobID] = clock
            disk_visits_per_job[jobID] = 0

            # Place new job at cpu queue
            exec_time = poisson_execution_time(54)
            CPUQueue.append([jobID, exec_time])
            per_cycle_jobs_list.append(jobID)

            # Define new arrival time and increment number of jobs
            time_of_next_arrival = clock + next_system_arrival()
            cpu_empty = False
            
            jobID += 1

        elif next_event_station(time_of_next_arrival, current_job_exec_start_time_cpu + current_job_exec_remaining_time_cpu, current_job_exec_start_time_disk + current_job_exec_remaining_time_disk) == "CPU":
            clock = current_job_exec_start_time_cpu + current_job_exec_remaining_time_cpu
            # Job leaves the system with p = 1/19
            random = U()
            if (random <= 1/19):
                time_of_departure_dict[current_job_ID_cpu] = clock+ 234
                if CPUQueue:
                    current_job_ID_cpu, current_job_exec_remaining_time_cpu = CPUQueue.popleft()
                    current_job_exec_start_time_cpu = clock
                elif not CPUQueue:
                    current_job_ID_cpu, current_job_exec_remaining_time_cpu, current_job_exec_start_time_cpu = None, None, None
                    cpu_last_empty_timestamp = clock
                    cpu_empty = True
                
            else:
                # Load job to the disk queue
                disk_visits_per_job[current_job_ID_cpu] += 1
                exec_time = poisson_execution_time(35)

                queue_DISK.append(
                    [current_job_ID_cpu, exec_time])

                # Load new job to the CPU
                if CPUQueue:
                    current_job_ID_cpu, current_job_exec_remaining_time_cpu = CPUQueue.popleft()
                    current_job_exec_start_time_cpu = clock
                elif not CPUQueue:
                    current_job_ID_cpu, current_job_exec_remaining_time_cpu, current_job_exec_start_time_cpu = None, None, None
                    cpu_last_empty_timestamp = clock
                    cpu_empty = True

        else:
            clock = current_job_exec_start_time_disk + current_job_exec_remaining_time_disk
            # Load job to the CPU queue
            exec_time = poisson_execution_time(54)

            CPUQueue.append([current_job_ID_disk, exec_time])

            # Load new job to the DISK
            if queue_DISK:
                current_job_ID_disk, current_job_exec_remaining_time_disk = queue_DISK.popleft()
                current_job_exec_start_time_disk = clock
            elif not queue_DISK:
                current_job_ID_disk, current_job_exec_remaining_time_disk, current_job_exec_start_time_disk = None, None, None
                disk_last_empty_timestamp = clock
                disk_empty = True

**Results**


In [ ]:
print("Average response time:",round(float(average(aggregate_response_time)/average(job_count)),3))
print("CPU utilization:",round(float(1 - average(cpu_unused_time_per_cycle)/average(cycle_duration)),3))
print("Disk utilization:",round(float(1 - average(disk_unused_time_per_cycle)/average(cycle_duration)),3))